#### Reinforcement Learning and Decision Making &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Homework #1

# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Planning in MDPs

## Description

You are given an $N$-sided die, along with a corresponding Boolean mask
vector, `is_bad_side` (i.e., a vector of ones and zeros). You can assume
that $1<N\leq30$, and the vector `is_bad_side` is also of size $N$ and
$1$ indexed (since there is no $0$ side on the die). The game of DieN is
played as follows:

1.  You start with $0$ dollars.

2.  At any time you have the option to roll the die or to quit the game.

    1.  **ROLL**:

        1.  If you roll a number not in `is_bad_side`, you receive that
            many dollars (e.g., if you roll the number $2$ and $2$ is
            not a bad side -- meaning the second element of the vector
            `is_bad_side` is $0$, then you receive $2$ dollars). Repeat
            step 2.

        2.  If you roll a number in `is_bad_side`, then you lose all the
            money obtained in previous rolls and the game ends.

    2.  **QUIT**:

        1.  You keep all the money gained from previous rolls and the
            game ends.

## Procedure

-   You will implement your solution using the `solve()` method
    in the code below.
    
-   Your return value should be the number of dollars you expect to
    win for a specific value of `is_bad_side`, if you follow an
    optimal policy. That is, what is the value of the optimal
    state-value function for the initial state of the game (starting
    with $0$ dollars)? Your answer must be correct to $3$ decimal
    places, truncated (e.g., 3.14159265 becomes 3.141).

-   To solve this problem, you will need to determine an optimal policy
    for the game of DieN, given a particular configuration of the die.
    As you will see, the action that is optimal will depend on your
    current bankroll (i.e., how much money you've won so far).

-   You can try solving this problem by creating an MDP of the game
    (states, actions, transition function, reward function, and assume a
    discount rate of $\gamma=1$) and then calculating the optimal
    state-value function.

## Resources

The concepts explored in this homework are covered by:

-   Lecture Lesson 1: Smoov & Curly's Bogus Journey

-   Chapter 3 (3.6 Optimal Policies and Optimal Value Functions) and
    Chapter 4 (4.3-4.4 Policy Iteration, Value Iteration) of
    http://incompleteideas.net/book/the-book-2nd.html

-   Chapters 1-2 of 'Algorithms for Sequential Decision Making', M.
    Littman, 1996

## Submission

-   The due date is indicated on the Canvas page for this assignment.
    Make sure you have your timezone in Canvas set to ensure the
    deadline is accurate.

-   Submit your finished notebook on Gradescope. Your grade is based on
    a set of hidden test cases. You will have unlimited submissions -
    only the last score is kept.

-   Use the template below to implement your code. We have also provided
    some test cases for you. If your code passes the given test cases,
    it will run (though possibly not pass all the tests) on Gradescope.

-   Gradescope is using *python 3.6.x* and *numpy==1.18.0*, and you can
    use any core library (i.e., anything in the Python standard library).
    No other library can be used.  Also, make sure the name of your
    notebook matches the name of the provided notebook.  Gradescope times
    out after 10 minutes.

In [5]:
#################
# DO NOT REMOVE
# Versions
# numpy==1.18.0
################
import numpy as np

class MDPAgent(object):
    def __init__(self):
        pass

    def is_state_roll_worthy(self, state, is_bad_side):
        x = np.array(is_bad_side)
        bad_sides = np.where(x == 1)[0] + 1
        good_sides = np.where(x == 0)[0] + 1
        return len(bad_sides) / len(is_bad_side) * (-state) + sum(
            [1 / len(is_bad_side) * side for side in good_sides]) > 0

    def find_reachable_states(self, state, is_bad_side):
        x = np.array(is_bad_side)
        good_sides = np.where(x == 0)[0] + 1
        return good_sides + state

    def solve(self, is_bad_side):
        if self.validate_die(is_bad_side):
            print(is_bad_side)
            initial_state = 0
            reachable_states_dict = dict()
            reachable_states_dict[initial_state] = True
            self.find_all_reachable_states(initial_state, is_bad_side, reachable_states_dict)
            return self.value_iteration(reachable_states_dict, is_bad_side)
        return 0.000

    def validate_die(self, is_bad_side):
        is_valid_die = True
        if len(is_bad_side) < 2 or len(is_bad_side) > 30:
            is_valid_die = False
        x = np.array(is_bad_side)
        good_sides = np.where(x == 0)[0] + 1
        if len(good_sides) == len(is_bad_side):
            is_valid_die = False
        b = [0, 1]
        diff = [e for e in is_bad_side if e not in b]
        if len(diff) > 0:
            is_valid_die = False
        return is_valid_die

    def actions(self):
        actions = list()
        actions.append('roll')
        actions.append('quit')
        return actions

    def value_iteration(self, reachable_states_dict, is_bad_side):
        sorted_reachable_states = dict()
        for i in sorted(reachable_states_dict.keys()):
            sorted_reachable_states[i] = reachable_states_dict[i]
        v = {}
        for s in sorted_reachable_states.keys():
            v[s] = 0

        def merge(list1, list2):
            merged_list = [(list1[k], list2[k]) for k in range(0, len(list1))]
            return merged_list

        def q(state, action):
            if action == 'roll':
                x = np.array(is_bad_side)
                bad_sides = np.where(x == 1)[0] + 1
                good_sides = np.where(x == 0)[0] + 1
                rs = self.find_reachable_states(state, is_bad_side)
                merge_t = merge(good_sides, rs)
                return len(bad_sides) / len(is_bad_side) * (-state) + sum(
                    [1 / len(is_bad_side) * (r + v[s]) for r, s in merge_t])
            return v[state]

        while True:
            new_v = {}
            for s in sorted_reachable_states.keys():
                if not sorted_reachable_states[s]:
                    new_v[s] = 0
                else:
                    new_v[s] = max(q(s, action) for action in self.actions())
            if max(abs(v[s] - new_v[s]) for s in sorted_reachable_states.keys()) < 1e-4:
                break
            v = new_v
        return v[0]

    def find_all_reachable_states(self, state, is_bad_side, reachable_states_dict):
        reachable_states = self.find_reachable_states(state, is_bad_side)
        for s in reachable_states:
            if s not in reachable_states_dict:
                if self.is_state_roll_worthy(s, is_bad_side) or state == 0:
                    reachable_states_dict[s] = True
                    self.find_all_reachable_states(s, is_bad_side, reachable_states_dict)
                else:
                    reachable_states_dict[s] = False

## Test cases

We have provided some test cases for you to help verify your implementation.

In [ ]:
## DO NOT MODIFY THIS CODE.  This code will ensure that your submission
## will work proberly with the autograder

import unittest

class TestDieNNotebook(unittest.TestCase):
    def test_case_1(self):
        agent = MDPAgent()
        np.testing.assert_almost_equal(
            agent.solve(is_bad_side=[1, 1, 1, 0, 0, 0]),
            2.583,
            decimal=3
        )
        
    def test_case_2(self):
        agent = MDPAgent()
        np.testing.assert_almost_equal(
            agent.solve(
                is_bad_side=[1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0]
            ),
            7.379,
            decimal=3
        )
        
    def test_case_3(self):
        agent = MDPAgent()

        np.testing.assert_almost_equal(
            agent.solve(
                is_bad_side=[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0]
            ),
            6.314,
            decimal=3
        )

unittest.main(argv=[''], verbosity=2, exit=False)